In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import time
import re
from lxml import html
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
simuwang_url = 'https://www.simuwang.com/user/option'
zixuan_url = 'https://www.simuwang.com/user/option'

In [ ]:
username = '13834596916'
password = 'zxcvbnm1'

fund_codes = ['SW4663', 'SXW323', 'SVY664', 'SW7404']

登陆排排网

In [ ]:
#options = webdriver.ChromeOptions()
#driver = webdriver.Chrome(options=options)         # 启动驱动器

options = webdriver.EdgeOptions()
driver = webdriver.Edge(options=options)
#options = webdriver.EdgeOptions()
#driver = webdriver.Edge(options=options)

# 谷歌浏览器 79和79版本后防止被检测
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})

# 私募排排网：
driver.get(simuwang_url)       # 加载网站

driver.find_element(By.XPATH,'//button[@class="comp-login-method comp-login-b2"]').click()      #点击账号密码登录
driver.find_element(By.XPATH,'//input[@name="username"]').send_keys(username)              # 输入账号
driver.find_element(By.XPATH,'//input[@type="password"]').send_keys(password)                 # 输入密码
driver.find_element(By.XPATH,'//span[@class="checkbox"]').click()                               # 勾选同意协议
time.sleep(30) # solved this bug by adding sleep time to make sure the page is loaded
driver.find_element(By.XPATH,'//button[@class="comp-login-btn"]').click()                       # 点击登录

搜索指定基金加入自选

In [ ]:
not_found = []

for fund in fund_codes:
    time.sleep(15)
    driver.find_element(By.XPATH,'//input[@id="nav_keyword"]').send_keys(fund)
    driver.find_element(By.XPATH,'//button[@class="comp-header-submit"]').click()
    time.sleep(30)

    try:
        driver.find_element(By.XPATH,'//span[@class="select"]').click()
    except:
        not_found.append(fund)
    
    time.sleep(5)
    if fund != fund_codes[-1]:
        driver.get(simuwang_url)

if len(not_found) > 0:
    funds = '、'.join(not_found)
    print(funds + ' 不在私募排排网中！')
    

In [ ]:
driver.get(zixuan_url)
time.sleep(15)

page = driver.page_source
soup = BeautifulSoup(page,'html.parser')

link_elements = soup.select("a[href]")

urls = []
for link_element in link_elements:
   url = link_element['href']
   if "https://dc.simuwang.com/product" in url:
      urls.append(url)

print(np.array(urls))

逐个读取数据

In [ ]:
def strat(soup):
    tags = [element.text for element in soup.select("div.line2-tag")]
    if '私募 ' in tags:
        tags.remove('私募 ')
    if '正在运作 ' in tags:
        tags.remove('正在运作 ')

    return '、'.join(tags)

def extract_percentage(string):

    pattern = r'\d+\.?\d*%'
    matches = re.findall(pattern, string)

    return matches

def output_result(data):
    if '--' in data:
        return '--'
    else:
        return extract_percentage(data) 

In [ ]:
prod_info = {k:[] for k in ['基金名称', '管理人', '策略', '今年来收益', '成立来收益',
                           '成立以来年化', '成立来回撤', '成立以来夏普', '基金状态', 
                           '成立日期', '基金经理', '备案编号', '公司管理规模', 
                           '今年以来排名']}

for url in urls:
    driver.get(url)
    time.sleep(30)
    page = driver.page_source
    soup = BeautifulSoup(page,'html.parser')

    # 获取基金名称
    prod_info['基金名称'].append(soup.select("h1.line1-title-name.ellipsis")[0].text) 
    prod_info['管理人'].append(soup.select("a.line4-baseinfo-value.blue")[0].text)
    prod_info['策略'].append(strat(soup))

    profit = soup.select("div.line3-common.flex-h-center")
    this_year_pf = profit[1].text
    all_time_pf  = profit[2].text
    prod_info['今年来收益'].append(output_result(this_year_pf))
    prod_info['成立来收益'].append(output_result(all_time_pf))

    # 其他数据
    data = list(soup.select("span.line4-baseinfo-value"))[1:]

    names = ['成立以来年化', '成立来回撤', '成立以来夏普', '基金状态', '成立日期', 
             '基金经理', '备案编号', '公司管理规模', '今年以来排名']

    for i in range(len(data)):
        prod_info[names[i]].append(data[i].text)

In [ ]:
prod_info = pd.DataFrame(prod_info)

with pd.ExcelWriter('prod_info.xlsx') as excel_writer:
    prod_info.to_excel(excel_writer, sheet_name='Sheet1', index=False)